In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import tiktoken
import pinecone
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings

##  using OPENAI-EMBEDDING-3-LARGE

load_dotenv()
token=os.getenv("GITHUB_API_TOKEN")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

endpoint = "https://models.inference.ai.azure.com"
model_name="gpt-4o-mini"
#embedding_model = "text-embedding-3-large"

def num_tokens_from_string(input_text: str, encoding_name: str) -> str:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(input_text))
    return num_tokens

embedding_model=OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_base=endpoint,
            openai_api_key=token
        )

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index = pc.Index(name='email')
# View index stats
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {'email_guide': {'vector_count': 7}},
 'total_vector_count': 7,
 'vector_type': 'dense'}

In [2]:
# Example similarity search
query = "How to write a referral request email to manager for applied job?"
query_embedding = embedding_model.embed_query(query)

In [3]:
## Search with a dense vector
searchresults = index.query(
    namespace="email_guide",
    vector=query_embedding,
    top_k=2,
    include_metadata=True
)

In [4]:
context = "\n\n".join([match['metadata']['text'] for match in searchresults['matches']])

In [5]:
print(context)

Hiring  Manager  –  Decision  Maker  
●  Goal :  Show  value  to  land  an  interview.  ●  Context :  Hiring  managers  own  the  role  and  care  about  team  fit  and  problem-solving.  
The
 
goal
 
is
 
to
 
subtly
 
pitch
 
your
 
value
 
and
 
get
 
on
 
their
 
radar.
 ●  Tone :  Professional,  solution-focused,  and  collaborative—respecting  their  authority  while  
showing
 
initiative.
 ●  Customizations :  •  Subject:  “Thoughts  on  Your  [Job  Title]  Opening”  •  Opener:  
References
 
their
 
team
 
or
 
role
 
directly.
 
•
 
Pitch:
 
Ties
 
your
 
work
 
to
 
their
 
likely
 
needs.
 
•
 
Request:
 
Positions
 
you
 
as
 
a
 
potential
 
team
 
asset.
 ●  Why  It  Works :  Ties  skills  to  their  needs,  respects  their  authority.  
Example :  
Subject:  Thoughts  on  Your  Data  Scientist  Opening  
Hi  John,  
I  hope  this  finds  you  well!  I’m  reaching  out  about  the  Data  Scientist  role  on  your  
team
 
at
 
[Company],
 
which
 
caught
 
my
 
eye
 
as

In [7]:
num_tokens_from_string(context, "cl100k_base")

1012

In [8]:
client = OpenAI(
    base_url=endpoint,
    api_key=token
)
final_prompt = f"Use the following context to answer:\n\n{context}\n\nQ: {query}\nA:"
chat_response = client.chat.completions.create(
    model=model_name,  
    messages=[
        {"role": "system", "content": "You are an expert in writing emails using context data, along with the user provided query."},
        {"role": "user", "content": final_prompt}
    ],
    temperature=0.7
)
final_email_response=chat_response.choices[0].message.content
print("vvvvvvvvvv  Generated Email Response  vvvvvvvvvvv")
print("-------------------------------")
print(final_email_response)
email_output_tokens=num_tokens_from_string(final_email_response, "o200k_base")

vvvvvvvvvv  Generated Email Response  vvvvvvvvvvv
-------------------------------
Subject: Request for Referral for [Job Title] Position

Hi [Manager's Name],

I hope this message finds you well! I recently applied for the [Job Title] position with [Company] and wanted to reach out to you regarding it. Given your extensive experience and insight into the team, I thought it would be valuable to connect.

I am genuinely excited about the opportunity to contribute to [specific team or project], especially with my background in [relevant skills or experiences]. For instance, in my previous role, I [briefly mention a relevant achievement or project that aligns with the team’s needs]. I believe this experience could be beneficial as your team addresses [specific challenges or goals they might have].

If you feel comfortable, I would greatly appreciate any support or referral you could provide to help me stand out in the hiring process. I am eager to bring my skills to [Company] and contribut